<a href="https://colab.research.google.com/github/Billy-24/my_mathematics_repo_temp/blob/main/Copie_de_la_version_definitive_du_Projet_wintage_version_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from google.colab import files
import io
uploaded = files.upload()
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# L'analyse exploratoire des données
data = pd.read_csv("/content/drive/MyDrive/winemag-data-130k-v2.csv")
data.head()

In [ ]:
# type de variable du dataset
data.info()

In [ ]:
data.winery .nunique()

In [ ]:
data.isnull().sum()

In [ ]:
# Supprimer les colonnes non pertinentes
data = data.drop(['Unnamed: 0','region_2', 'description', 'taster_twitter_handle'], axis=1)
data.head()

In [ ]:
data.describe(include="all")

In [ ]:
# vérification des valeurs manquantes dans le dataset
plt.figure(figsize=(20,10))
sns.heatmap(data.isna(), cbar=False)

In [ ]:
# Verification des doublons
duplicatas = data[data.duplicated()]
print("Enregistrements Dupliqués :", duplicatas.shape)

In [ ]:
#supprimons les doublons
data.drop_duplicates(inplace=True)

In [ ]:
data.info()

In [ ]:
# L'ensemble de données présente toutes les variables prédictives comme catégoriques, à l'exception du « prix ». La variable dépendante est « points ».cat_data=[]
# Sectionner les catégories des valeur numériques
cat_data=[]
num_data=[]
for i,c in enumerate(data.dtypes):
  if c==object:
    cat_data.append(data.iloc[:,i])
  else:
    num_data.append(data.iloc[:,i])
cat_data=pd.DataFrame(cat_data)
cat_data

In [ ]:
# Transposer les valeurs manquantes sectionnées
cat_data=[]
num_data=[]
for i,c in enumerate(data.dtypes):
  if c==object:
    cat_data.append(data.iloc[:,i])
  else:
    num_data.append(data.iloc[:,i])
cat_data=pd.DataFrame(cat_data).transpose()
num_data=pd.DataFrame(num_data).transpose()
cat_data

In [ ]:
# pour les valeurs manquantes dans les variables cathégoriques, on doit les remplacer par le Mode
cat_data=cat_data.apply(lambda x:x.fillna(x.value_counts().index[0]))
cat_data.isnull().sum().any()

In [ ]:
# Analyse des valeurs numériques
data[['points','price']].describe()

In [ ]:
# Analyser la correlation entre price et points
data[['price','points']].corr(method = 'pearson')

In [ ]:
# Carte de corrélation
plt.figure(figsize=(2,2),)
correlacion = data.corr()
sns.heatmap(correlacion,annot=True)

In [ ]:
# Suite à ce résultat 'il existe une corrélation positive modérée entre 'price' et 'points' dans vos données. Cela suggère qu'il y a une tendance générale selon laquelle des vins avec des prix plus élevés ont tendance à obtenir des points plus élevés, et vice versa. Cependant, d'autres facteurs peuvent également influencer la notation des vins, car la corrélation n'est pas parfaite.

In [ ]:
# Analyse country - points
data[['points','country']].describe()

In [ ]:
top_countries = data.groupby(['country'])['points'].mean().sort_values(ascending=False).head(10)
print(top_countries)

In [ ]:
# Analyse des variables numériques
# Analyse univariée des valeurs numériques : Créez un nuage de points avec 'price' sur l'axe des x
plt.scatter(range(len(data['price'])), data['price'])
plt.show()

In [ ]:
# Étape 1 : Supprimer la valeur aberrante dans 'price'
seuil_aberrant = 3200
num_data = num_data[num_data['price'] <= seuil_aberrant]

In [ ]:
# Nuage de points après suppression de la valeur aberrante
plt.scatter(range(len(num_data['price'])), num_data['price'])
plt.title('Nuage de points après suppression de la valeur aberrante')
plt.xlabel('Index')
plt.ylabel('Price')
plt.show()

In [ ]:
# Étape 2 : Remplacer les valeurs manquantes dans 'price' par la moyenne
moyenne_price = num_data['price'].mean()
num_data['price'].fillna(moyenne_price, inplace=True)
num_data

In [ ]:
 #Un test pour voir si on retrouve encore des NAN dans le dataset
plt.figure(figsize=(20,10))
sns.heatmap(num_data.isna(), cbar=False)

In [ ]:
# Fusionner les ensembles de données en fonction des colonnes
combined_data = pd.concat([cat_data, num_data], axis=1)

# Afficher le résultat
combined_data


In [ ]:
# Remplacer les NaN par la moyenne dans les colonnes 'points' et 'price'
combined_data['points'].fillna(combined_data['points'].mean(), inplace=True)
combined_data['price'].fillna(combined_data['price'].mean(), inplace=True)
# Afficher le résultat
combined_data

In [ ]:
 #Un test pour voir si on retrouve encore des NAN dans le dataset
plt.figure(figsize=(20,10))
sns.heatmap(combined_data.isna(), cbar=False)

In [ ]:
### Training pipeline ###
print("--- Training pipeline ---")
print()

# Encoding categorical features and standardizing numeric features

print("#### X_train BEFORE preprocessing ####")
display(combined_data.head())
print()

print("Encoding categorical features and standardizing numerical features...")
## First let's import libraries
## StandardScaler to scale data (i.e apply Z-score)
## OneHotEncoder to encode categorical variables
numeric_features = ['points', '	price'] # Choose which column index we are going to scale
numeric_transformer = StandardScaler()
categorical_features = ['country',	'designation',	'province',	'region_1',	'taster_name', 'variety', 'winery	'] # Choose which column index we are going to encode
categorical_transformer = OneHotEncoder(drop="first")
# Apply ColumnTransformer to create a pipeline that will apply the above preprocessing
feature_encoder = ColumnTransformer(transformers=[('cat', categorical_transformer, categorical_features), ('num', numeric_transformer, numeric_features)])


In [ ]:
# pipeline de prétraitement des données qui encode les variables catégorielles et normalise les variables numériques, puis applique cette pipeline aux données d'entraînement. Cela prépare les données pour être utilisées dans un modèle d'apprentissage automatique.

In [ ]:
# Encoding categorical features and standardizing numeric features

print("#### X_train BEFORE preprocessing ####")
display(combined_data.head())
print()

print("Encoding categorical features and standardizing numerical features...")
## First let's import libraries
## StandardScaler to scale data (i.e apply Z-score)
## OneHotEncoder to encode categorical variables
numeric_features = ['points', 'price']  # Choose which column index we are going to scale
numeric_transformer = StandardScaler()
categorical_features = ['country', 'designation', 'province', 'region_1', 'taster_name', 'variety', 'winery']  # Choose which column index we are going to encode
categorical_transformer = OneHotEncoder(drop="first")

# Apply ColumnTransformer to create a pipeline that will apply the above preprocessing
feature_encoder = ColumnTransformer(transformers=[('cat', categorical_transformer, categorical_features), ('num', numeric_transformer, numeric_features)])

# Apply the pipeline to combined_data and store the result in X_train_preprocessed
X_train_preprocessed = feature_encoder.fit_transform(combined_data)

#Display (the first few rows of the preprocessed data)
print("#### X_train AFTER preprocessing ####")
display(X_train_preprocessed.toarray())


In [ ]:
X_train_preprocessed.toarray().shape

In [ ]:
# Apply ColumnTransformer to create a pipeline that will apply the above preprocessing
feature_encoder = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(), categorical_features),  # Remove drop="first"
        ('num', numeric_transformer, numeric_features)
    ]
)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import pandas as pd

# Sur la base de notre DataFrame
X = combined_data[['country', 'designation', 'province', 'region_1', 'taster_name', 'variety', 'winery', 'price']]  # Je n'inclus pas 'points' ici car c'est la variable cible
y = combined_data['points']


# Définissez les transformations
numeric_features = ['price']
numeric_transformer = StandardScaler()
categorical_features = ['country', 'designation', 'province', 'region_1', 'taster_name', 'variety', 'winery']
categorical_transformer = OneHotEncoder(drop="first")

# Appliquer le ColumnTransformer à vos données d'entraînement
feature_encoder = ColumnTransformer(transformers=[('cat', categorical_transformer, categorical_features),
 ('num', numeric_transformer, numeric_features)])
X_encoded = feature_encoder.fit_transform(X)

# Obtention des noms de colonnes après le prétraitement
feature_names_out = feature_encoder.get_feature_names_out(input_features=categorical_features + numeric_features)

# Affichez les dix premiers noms de colonnes
print(feature_names_out[:10])


In [ ]:
# Afficher les noms de colonnes obtenus après la transformation
print(feature_names_out)

In [ ]:
# Divisons l'ensemble de données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42)

# Affichons les dimensions des ensembles d'entraînement et de test
print("Dimensions de l'ensemble d'entraînement X :", X_train.shape)
print("Dimensions de l'ensemble de test X :", X_test.shape)
print("Dimensions de l'ensemble d'entraînement y :", y_train.shape)
print("Dimensions de l'ensemble de test y :", y_test.shape)

In [ ]:
# ça c'est pour le radom forest c'est à modifier pour mettre la regression linéaire
#Créez et entraînez le modèle de régression linéaire
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor()
model.fit(X_train, y_train)

In [ ]:
# Créez et entraînez le modèle de régression linéaire
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder
model = LinearRegression()
model.fit(X_train, y_train)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, median_absolute_error
# Faire des prédictions sur l'ensemble de test
y_pred = model.predict(X_test)
# Calculer les métriques de performance sur l'ensemble de test
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
medae = median_absolute_error(y_test, y_pred)
rmse = np.sqrt(mse)
# Afficher les métriques de performance sur l'ensemble de test
print(f'MAE test : {mae}')
print(f'MedAE test : {medae}')
print(f'MSE test : {mse}')
print(f'RMSE test : {rmse}')
print(f'R² test : {r2}')


 # Interprétation des métriques pour le modèle de régression
## Mean Absolute Error (MAE) test : 0.6155
"En moyenne, les prédictions du modèle diffèrent d'environ 0.6155 points de la vérité terrain. Cela signifie que, en moyenne, le modèle a une erreur d'environ 0.6155 points par rapport aux valeurs réelles.

## Median Absolute Error (MedAE) test : 0.4869
"La moitié des prédictions du modèle sur l'ensemble de test diffèrent de 0.4869 points ou moins de la vérité terrain. Cela suggère que le modèle a une précision médiane relativement bonne.

## Mean Squared Error (MSE) test : 0.6454
"En moyenne, l'erreur quadratique entre les prédictions et les valeurs réelles sur l'ensemble de test est d'environ 0.6454 points. Cela indique que, en général, les prédictions s'écartent peu des valeurs réelles.

## Root Mean Squared Error (RMSE) test : 0.8034
"La racine carrée du MSE est d'environ 0.8034 points. Une valeur faible de RMSE montre que le modèle fait de bonnes prédictions en termes d'écart absolu moyen par rapport aux valeurs réelles.

## R-squared (R²) test : 0.3538
"Le R² est d'environ 0.3538, ce qui signifie que le modèle explique environ 35.38 % de la variance dans la variable cible. Bien que le modèle apporte une certaine explication, il reste de la variance non expliquée.

# En conclusion, bien que le modèle ait une certaine capacité prédictive, il y a encore de la place pour l'amélioration, surtout si vous cherchez à réduire davantage l'erreur de prédiction

In [ ]:
# Visualisation des prédictions par rapport aux valeurs réelles
# Diagramme de Dispersion
plt.figure(figsize=(8, 6))
sns.scatterplot(x=y_test, y=y_pred)
plt.title('Diagramme de Dispersion')
plt.xlabel('Valeurs Réelles')
plt.ylabel('Prédictions')
plt.show()

In [ ]:
# Histogramme des Erreurs
plt.figure(figsize=(8, 6))
sns.histplot(errors.to_numpy(), bins=50, kde=True)
plt.title('Histogramme des Erreurs')
plt.xlabel('Erreurs')
plt.ylabel('Fréquence')
plt.show()

In [ ]:
# Position du pic: Nous constatons que l'histogramme est centré autour de zéro, cela signifie que notre modèle a tendance à faire des prédictions proches des valeurs réelles.

In [ ]:
# Courbe de Régression
plt.figure(figsize=(8, 6))
sns.regplot(x=y_test, y=y_pred, scatter_kws={'s': 20, 'alpha': 0.3})
plt.title('Courbe de Régression')
plt.xlabel('Valeurs Réelles')
plt.ylabel('Prédictions')
plt.show()

In [ ]:
# On observe une dispersion importante autour de la ligne de régression ce qui indique une variabilité élevée dans les prédictions.

In [ ]:
# Diagramme de Résidus
plt.figure(figsize=(8, 6))
sns.residplot(x=y_test, y=errors, lowess=True, scatter_kws={'s': 20, 'alpha': 0.3})
plt.title('Diagramme de Résidus')
plt.xlabel('Valeurs Réelles')
plt.ylabel('Résidus')
plt.show()